1. Setup Environment 🔽

This cell downloads the initial setup.py script from your fork and then executes it, instructing the setup process to pull all subsequent project files from your repository.

In [1]:
# @title 1. Setup Environment (Final Path Correction)
import os
import sys
from pathlib import Path

# --- DYNAMIC PLATFORM DETECTION & PATH SETUP ---
# This logic ensures the notebook and the scripts it runs agree on the correct home directory.
def get_platform_home():
    """Determines the correct project HOME based on the runtime environment."""
    if 'google.colab' in sys.modules:
        print("Platform: Google Colab")
        return Path('/content')
    if os.path.exists('/kaggle'):
        print("Platform: Kaggle")
        return Path('/kaggle/working')
    if os.environ.get('LIGHTNING_AI') or os.path.exists('/teamspace'):
        print("Platform: Lightning AI")
        base_path = Path('/teamspace/studios/this_studio')
        if not base_path.exists():
            base_path = Path.home() / 'workspace'
        return base_path
    # Fallback for local or unrecognized environments
    print("Platform: Local / Unknown")
    return Path.cwd()

# Define the single source of truth for the project's home directory for this entire session
PROJECT_HOME = get_platform_home()
print(f"✅ Project HOME directory set to: {PROJECT_HOME}")

# --- Configuration ---
lang = 'en'
branch = 'main'
fork_repo = 'drf0rk/AnxietyLightning'

# --- Script Execution ---
anxety_path = PROJECT_HOME / 'ANXETY'
scripts_dir_temp = anxety_path / 'scripts'
setup_script_path = scripts_dir_temp / 'setup.py'

os.makedirs(scripts_dir_temp, exist_ok=True)

setup_url = f'https://raw.githubusercontent.com/{fork_repo}/{branch}/scripts/setup.py'
print(f"🔄 Downloading setup script from {fork_repo}...")
!curl -sLo {setup_script_path} {setup_url}
print("✅ Setup script downloaded.")

# Run the setup script to detect the platform and create settings.json
%run {setup_script_path} --lang={lang} --branch={branch} --fork={fork_repo}

# --- Set Definitive Paths for Subsequent Cells ---
print("\n✅ Setting up definitive paths for the notebook session...")
modules_path = anxety_path / 'modules'
if str(modules_path) not in sys.path:
    sys.path.insert(0, str(modules_path))

import json_utils as js
settings_path = anxety_path / 'settings.json'

# --- THIS IS THE CORRECTED LOGIC ---
# Read the base 'scr_path' (e.g., /content/ANXETY) and then append '/scripts' to it.
base_scr_path = Path(js.read(settings_path, 'ENVIRONMENT.scr_path'))
scripts_dir = base_scr_path / 'scripts'
# --- END CORRECTION ---

print(f"✔️ Scripts directory for this session is set to: {scripts_dir}")


✅ Setting up definitive paths for the notebook session...
✔️ Scripts directory for this session is set to: /content/ANXETY/scripts


In [16]:
# @title Cell 1.5: Asset Downloader (Final Version)
import ipywidgets as widgets
from IPython.display import display, HTML
import requests
import json
import re
from pathlib import Path
import runpy

# --- 1. SETUP AND PATHS ---
print("✅ Initializing Asset Downloader...")
cwd = Path.cwd()
project_dir_name = 'ANXETY'
if cwd.name == project_dir_name:
    ANXETY_ROOT = cwd
else:
    ANXETY_ROOT = cwd / project_dir_name

if not ANXETY_ROOT.is_dir():
    raise FileNotFoundError(f"FATAL: Could not find project root at '{ANXETY_ROOT}'")

models_data_path = ANXETY_ROOT / 'scripts/_models-data.py'
xl_models_data_path = ANXETY_ROOT / 'scripts/_xl-models-data.py'
loras_data_path = ANXETY_ROOT / 'scripts/_loras-data.py'
print("   - Paths configured.")

# --- 2. CORE LOGIC ---

def get_civitai_model_info(model_id):
    """Fetches model details from Civitai API."""
    api_url = f"https://civitai.com/api/v1/models/{model_id}"
    try:
        response = requests.get(api_url, timeout=10)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data for model ID {model_id}: {e}")
        return None

def parse_civitai_url(url):
    """Extracts model ID from a Civitai URL."""
    match = re.search(r'models/(\d+)', url)
    return match.group(1) if match else None

def get_hf_filename(url):
    """Extracts a filename from a Hugging Face URL."""
    return url.split('/')[-1].split('?')[0]

def append_to_py_dict_file(file_path, dict_name, new_key, new_value_dict):
    """Safely appends a new entry to a dictionary in a Python file."""
    if not file_path.exists():
        with file_path.open('w', encoding='utf-8') as f:
            f.write(f"{dict_name} = {{\n")
            f.write(f'    "{new_key}": {json.dumps(new_value_dict)},\n')
            f.write("}\n")
        print(f"   - Created and added '{new_key}' to {file_path.name}")
        return

    content = file_path.read_text(encoding='utf-8')
    dict_pattern = re.compile(f"^{dict_name}\\s*=\\s*{{", re.MULTILINE)
    match = dict_pattern.search(content)

    if not match:
        print(f"   - Could not find dictionary '{dict_name}' in {file_path.name}. Appending to end.")
        with file_path.open('a', encoding='utf-8') as f:
            f.write(f"\n{dict_name} = {{\n")
            f.write(f'    "{new_key}": {json.dumps(new_value_dict)},\n')
            f.write("}\n")
        return

    start_index = match.end()
    open_braces = 1
    end_index = -1
    for i, char in enumerate(content[start_index:]):
        if char == '{': open_braces += 1
        elif char == '}':
            open_braces -= 1
            if open_braces == 0:
                end_index = start_index + i
                break

    if end_index == -1:
        print(f"   - Could not find closing brace for '{dict_name}' in {file_path.name}. Cannot update.")
        return

    file_globals = runpy.run_path(str(file_path))
    if new_key in file_globals.get(dict_name, {}):
        print(f"   - Item '{new_key}' already exists in {dict_name}. Skipping.")
        return

    new_entry_str = f'    "{new_key}": {json.dumps(new_value_dict)},\n'
    new_content = content[:end_index] + new_entry_str + content[end_index:]
    file_path.write_text(new_content, encoding='utf-8')
    print(f"   - Successfully added '{new_key}' to {dict_name} in {file_path.name}")

def append_to_nested_lora_file(file_path, sub_dict_name, new_key, new_value_dict):
    """Specialized function to append to a nested dictionary in the LoRA file."""
    if not file_path.exists():
        print(f"   - LoRA file not found at {file_path}. Cannot create nested structure automatically.")
        return

    content = file_path.read_text(encoding='utf-8')
    dict_pattern = re.compile(f'"{sub_dict_name}"\\s*:\\s*{{', re.DOTALL)
    match = dict_pattern.search(content)

    if not match:
        print(f"   - Could not find sub-dictionary '{sub_dict_name}' in {file_path.name}. Manual addition required.")
        return

    start_index = match.end()
    open_braces = 1
    end_index = -1
    for i, char in enumerate(content[start_index:]):
        if char == '{': open_braces += 1
        elif char == '}':
            open_braces -= 1
            if open_braces == 0:
                end_index = start_index + i
                break

    if end_index == -1:
        print(f"   - Could not find closing brace for '{sub_dict_name}' in {file_path.name}. Cannot update.")
        return

    file_globals = runpy.run_path(str(file_path))
    if new_key in file_globals.get('lora_data', {}).get(sub_dict_name, {}):
        print(f"   - Item '{new_key}' already exists in {sub_dict_name}. Skipping.")
        return

    new_entry_str = f'        "{new_key}": [{json.dumps(new_value_dict)}],\n'
    new_content = content[:end_index] + new_entry_str + content[end_index:]
    file_path.write_text(new_content, encoding='utf-8')
    print(f"   - Successfully added '{new_key}' to {sub_dict_name} in {file_path.name}")


def process_links(button):
    """Main function triggered by the button click."""
    output_widget.clear_output()
    with output_widget:
        print("🚀 Starting processing...")
        urls = [url.strip() for url in links_textarea.value.splitlines() if url.strip()]
        if not urls:
            print("❌ No URLs provided. Please paste URLs into the text box.")
            return

        for url in urls:
            print(f"\nProcessing URL: {url}")

            if 'civitai.com/models/' in url and 'modelVersionId=' in url:
                model_version_id = re.search(r'modelVersionId=(\d+)', url).group(1)
                download_url = f"https://civitai.com/api/download/models/{model_version_id}"
            else:
                download_url = url

            new_asset = {"url": download_url}
            asset_name = ""
            target_path = None
            target_dict = ""

            if 'civitai.com' in url:
                model_id = parse_civitai_url(url)
                if not model_id:
                    print(f"   - Not a valid Civitai model URL. Skipping.")
                    continue

                info = get_civitai_model_info(model_id)
                if not info: continue

                asset_name = info.get('name', f"civitai_{model_id}")
                asset_type = info.get('type', 'Unknown')

                # --- START OF SUPER-ROBUST SDXL CHECK ---
                base_model_str = info.get('baseModel', '').lower()
                model_name_str = info.get('name', '').lower()
                tags = [tag.lower() for tag in info.get('tags', [])]
                version_name_str = ''
                if info.get('modelVersions') and len(info['modelVersions']) > 0:
                    version_name_str = info['modelVersions'][0].get('name', '').lower()

                is_sdxl = 'sdxl' in base_model_str or 'sdxl' in model_name_str or 'sdxl' in version_name_str or 'sdxl' in tags
                is_pony = 'pony' in base_model_str or 'pony' in model_name_str or 'pony' in version_name_str or 'pony' in tags

                base_model = "SDXL" if (is_sdxl or is_pony) else "SD 1.5"
                # --- END OF SUPER-ROBUST SDXL CHECK ---

                new_asset['name'] = get_hf_filename(info['modelVersions'][0]['files'][0]['name'])

                if asset_type == 'Checkpoint':
                    if base_model == "SDXL":
                        target_path, target_dict = xl_models_data_path, 'sdxl_models_data'
                    else:
                        target_path, target_dict = models_data_path, 'sd15_model_data'
                    append_to_py_dict_file(target_path, target_dict, asset_name, new_asset)

                elif asset_type in ['LORA', 'LoCon']:
                    sub_dict = "sdxl_loras" if base_model == "SDXL" else "sd15_loras"
                    append_to_nested_lora_file(loras_data_path, sub_dict, asset_name, new_asset)

                elif asset_type == 'VAE':
                     if base_model == "SDXL":
                        target_path, target_dict = xl_models_data_path, 'sdxl_vae_data'
                     else:
                        target_path, target_dict = models_data_path, 'sd15_vae_data'
                     append_to_py_dict_file(target_path, target_dict, asset_name, new_asset)
                else:
                    print(f"   - Unsupported Civitai asset type: {asset_type}. Skipping.")
                    continue

            elif 'huggingface.co' in url:
                asset_name = get_hf_filename(url)
                new_asset['name'] = asset_name
                print("   - Hugging Face link detected. Assuming SD1.5 Checkpoint.")
                append_to_py_dict_file(models_data_path, 'sd15_model_data', asset_name, new_asset)

            else:
                print("   - URL is not from Civitai or Hugging Face. Skipping.")
                continue

        print("\n🏁 Processing complete. You may now run the next cell to see your new models in the GUI.")

# --- 3. WIDGETS AND DISPLAY ---
links_textarea = widgets.Textarea(placeholder='Paste one Civitai or Hugging Face URL per line...', layout=Layout(width='90%', height='200px'))
process_button = widgets.Button(description='Add Models to Library', button_style='success', layout=Layout(width='200px'))
output_widget = widgets.Output()

process_button.on_click(process_links)

display(HTML("<h2>Step 1: Add New Models & LoRAs</h2><p>Paste URLs below to add them to the selection GUI in the next step. The script will attempt to automatically categorize them.</p>"))
display(widgets.VBox([links_textarea, process_button, output_widget]))
print("✅ Asset Downloader Ready.")

✅ Initializing Asset Downloader...
   - Paths configured.


✅ Asset Downloader Ready.


2. Widgets 🔽

This cell will load the interactive widgets. Since setup.py (executed in the previous cell) has already downloaded the patched widgets-en.py (or widgets-ru.py) to scripts_dir, this command will now use your modified version with the LoRA dropdown.

In [14]:
# @title 2 File Selection

from IPython.display import display, HTML
from widget_factory import WidgetFactory
from webui_utils import update_current_webui
import json_utils as js
import ipywidgets as widgets
from ipywidgets import Layout
from pathlib import Path
import os
import re

# --- Constants and Platform-Aware Pathing ---
cwd = Path.cwd()
project_dir_name = 'ANXETY'
if cwd.name == project_dir_name:
    ANXETY_ROOT = cwd
else:
    ANXETY_ROOT = cwd / project_dir_name

if not ANXETY_ROOT.is_dir():
    raise FileNotFoundError(f"FATAL: Could not find project root at '{ANXETY_ROOT}'")

SETTINGS_PATH = ANXETY_ROOT / 'settings.json'
SCRIPTS = ANXETY_ROOT / 'scripts'
CSS = ANXETY_ROOT / 'CSS'
JS = ANXETY_ROOT / 'JS'
widgets_css = CSS / 'main-widgets.css'
widgets_js = JS / 'main-widgets.js'

# --- Helper Functions ---
factory = WidgetFactory()

def read_data_keys(file_path, data_key_in_file, prefixes=['none']):
    local_vars = {}
    if not file_path.exists():
        return prefixes
    with open(file_path, 'r', encoding='utf-8') as f:
        try:
            exec(f.read(), {}, local_vars)
        except Exception as e:
            return prefixes
    data_dict = local_vars.get(data_key_in_file, {})
    return prefixes + [f"{i+1}. {name}" for i, name in enumerate(data_dict.keys())]

def read_lora_keys_by_type(file_path):
    local_vars, sd15_keys, sdxl_keys = {}, [], []
    if not file_path.exists():
        return sd15_keys, sdxl_keys
    with open(file_path, 'r', encoding='utf-8') as f:
        exec(f.read(), {}, local_vars)
    lora_main_dict = local_vars.get('lora_data', {})
    sd15_keys = list(lora_main_dict.get('sd15_loras', {}).keys())
    sdxl_keys = list(lora_main_dict.get('sdxl_loras', {}).keys())
    return [f"{i+1}. {name}" for i, name in enumerate(sd15_keys)], \
           [f"{i+1}. {name}" for i, name in enumerate(sdxl_keys)]

# --- Widget Creation ---
model_list = read_data_keys(SCRIPTS / '_models-data.py', 'sd15_model_data')
XL_model_list = read_data_keys(SCRIPTS / '_xl-models-data.py', 'sdxl_models_data')
vae_list = read_data_keys(SCRIPTS / '_models-data.py', 'sd15_vae_data', ['none', 'ALL'])
XL_vae_list = read_data_keys(SCRIPTS / '_xl-models-data.py', 'sdxl_vae_data', ['none', 'ALL'])
cnet_list = read_data_keys(SCRIPTS / '_models-data.py', 'controlnet_list', ['none', 'ALL'])
XL_cnet_list = read_data_keys(SCRIPTS / '_xl-models-data.py', 'controlnet_list', ['none', 'ALL'])
sd15_lora_list, sdxl_lora_list = read_lora_keys_by_type(SCRIPTS / '_loras-data.py')

XL_models_widget = factory.create_checkbox(description='XL', value=False, class_names='sdxl')
model_widget = factory.create_select_multiple(description='Models:', options=model_list)
inpainting_model_widget = factory.create_checkbox(description='Inpainting', value=False, class_names='inpaint')
vae_widget = factory.create_select_multiple(description='VAE:', options=vae_list)
lora_widget = factory.create_select_multiple(description='LoRA:', options=sd15_lora_list)
controlnet_widget = factory.create_select_multiple(description='ControlNet:', options=cnet_list)

webui_options = ['A1111', 'Forge', 'ReForge', 'Classic', 'ComfyUI', 'SD-UX']
webui_selection = {
    'A1111': '--xformers --no-half-vae --enable-insecure-extension-access', 'Forge': '--xformers --forge-ref-a',
    'ReForge': '--xformers --reforge-ref-a', 'ComfyUI': '--windows-standalone-build', 'Classic': '', 'SD-UX': ''
}
latest_webui_widget = factory.create_checkbox('Update WebUI', False)
latest_extensions_widget = factory.create_checkbox('Update Extensions', False)
change_webui_widget = factory.create_dropdown('WebUI:', webui_options)
commandline_arguments_widget = factory.create_text('Arguments:', '')
GDrive_button = factory.create_button(description='Mount GDrive', class_names='gdrive-btn')
save_button = factory.create_button(description='Save settings', class_names='button_save')

# --- Callbacks ---
def on_xl_change(change):
    is_xl = change.new
    model_widget.options = XL_model_list if is_xl else model_list
    vae_widget.options = XL_vae_list if is_xl else vae_list
    controlnet_widget.options = XL_cnet_list if is_xl else cnet_list
    lora_widget.options = sdxl_lora_list if is_xl else sd15_lora_list

def on_webui_change(change):
    commandline_arguments_widget.value = webui_selection.get(change.new, '')

def on_save_click(button):
    save_settings()
    all_ui_components = [top_container, models_container, GDrive_button, save_button]
    factory.close(all_ui_components, class_names='hide')

# --- Settings ---
SETTINGS_KEYS = [
    'XL_models', 'model', 'inpainting_model', 'vae', 'lora', 'controlnet',
    'latest_webui', 'latest_extensions', 'change_webui', 'commandline_arguments',
]
def save_settings():
    widget_values = {key: globals()[f"{key}_widget"].value for key in SETTINGS_KEYS}
    js.save(str(SETTINGS_PATH), 'WIDGETS', widget_values)
    js.save(str(SETTINGS_PATH), 'mountGDrive', getattr(GDrive_button, 'toggle', False))
    update_current_webui(change_webui_widget.value)

def load_settings():
    if js.key_exists(str(SETTINGS_PATH), 'WIDGETS'):
        widget_data = js.read(str(SETTINGS_PATH), 'WIDGETS')
        for key in SETTINGS_KEYS:
            if key in widget_data and f"{key}_widget" in globals(): globals()[f"{key}_widget"].value = widget_data.get(key)
    GDrive_button.toggle = js.read(str(SETTINGS_PATH), 'mountGDrive', False)
    GDrive_button.add_class('active') if GDrive_button.toggle else GDrive_button.remove_class('active')

# --- Display ---
factory.load_css(widgets_css)

XL_models_widget.observe(on_xl_change, names='value')
change_webui_widget.observe(on_webui_change, names='value')
save_button.on_click(on_save_click)

webui_box = factory.create_vbox([change_webui_widget, commandline_arguments_widget, factory.create_hbox([latest_webui_widget, latest_extensions_widget])], 'box_webui')
settings_box = factory.create_vbox([factory.create_html('Settings', 'header'), factory.create_hbox([XL_models_widget, inpainting_model_widget])], 'box_settings')
top_container = factory.create_hbox([webui_box, settings_box], 'container_webui')
models_container = factory.create_vbox([model_widget, vae_widget, lora_widget, controlnet_widget], 'container_models')

display(top_container, models_container, GDrive_button, save_button)

load_settings()
js_content = ""
if widgets_js.exists():
    with open(widgets_js, 'r', encoding='utf-8') as f:
        js_content = f.read()
display(HTML(f"<script>{js_content}</script>"))

Button(description='Mount GDrive', style=ButtonStyle(), _dom_classes=('gdrive-btn',))

Button(description='Save settings', style=ButtonStyle(), _dom_classes=('button_save',))

3. Downloading 🔽

This cell initiates the downloading process for models, VAEs, extensions, and other necessary components. It will use the patched downloading-en.py (or downloading-ru.py) from your fork, ensuring downloads go to your centralized model storage location.

In [ ]:
# @title 2. File Selection
# This cell uses the 'scripts_dir' and 'lang' variables defined in Cell 1.
from IPython.display import display, HTML

# The f-string constructs the full path to the script.
run_path = f"{scripts_dir}/{lang}/widgets-{lang}.py"

print(f"✅ Loading interactive widgets from: {run_path}")
%run {run_path}

✅ Attempting to run script from: /content/ANXETY/scripts/en/downloading-en.py


ImportError: cannot import name 'Manager' from 'modules.Manager' (/content/ANXETY/modules/Manager.py)

4. Start 🔽

This cell launches the Stable Diffusion WebUI. It will execute the patched launch.py script from your fork, applying any platform-specific optimizations and arguments you've included.

In [ ]:
# @title 4. Start (Corrected Path Logic)
from pathlib import Path
import sys

# Add the modules path to ensure json_utils can be imported
anxety_path = Path.home() / 'ANXETY'
modules_path = anxety_path / 'modules'
if str(modules_path) not in sys.path:
    sys.path.insert(0, str(modules_path))

import json_utils as js

# Read the correct scripts_dir path that was saved by setup.py
settings_path = anxety_path / 'settings.json'
scripts_dir = Path(js.read(settings_path, 'ENVIRONMENT.scr_path'))

print(f"✅ Running launch script from: {scripts_dir}")
# Launch the WebUI. The -l flag provides more detailed logging for tunnels.
%run {scripts_dir}/launch.py -l

Utilities
5. Run Cleanup Utility 🔽

This cell runs the new Cleanup Utility GUI script, allowing you to manage your environment. This script is downloaded from your fork via the initial setup.

In [ ]:
# @title 5. Run Cleanup Utility

from pathlib import Path

# This cell is now platform-agnostic.
scripts_dir = Path.home() / 'ANXETY' / 'scripts'

# Run the cleanup utility GUI
%run {scripts_dir}/auto-cleaner.py

In [ ]:
from pathlib import Path
import shutil
import os
import sys

# --- Start of Directory Management ---
BASE_DIR = Path("/teamspace/studios/this_studio")
if os.getcwd() != str(BASE_DIR):
    print(f"🔄 Changing directory from {os.getcwd()} to {BASE_DIR}")
    os.chdir(BASE_DIR)
# --- End of Directory Management ---

# --- VERY IMPORTANT WARNING ---
# This cell will DELETE almost everything in your Lightning AI instance.
# It will only preserve this notebook file and 'main.py'.
# Ensure you understand what is being deleted before running.
# This operation is IRREVERSIBLE.

print("!!! DANGER: YOU ARE ABOUT TO DELETE ALMOST ALL FILES !!!")
print("!!! PLEASE READ CAREFULLY BEFORE PROCEEDING !!!")
print("\nThis cell will delete all folders and files in your current studio instance, EXCEPT:")
print(" - This notebook file (e.g., 'LightningAnxiety (1) (1) (2).ipynb')")
print(" - The 'main.py' file (if it exists in the root)")
print("\nTHIS WILL REQUIRE YOU TO RERUN THE ENTIRE NOTEBOOK FROM THE FIRST CELL FOR A FRESH START.")
print("If you have any custom files you wish to keep, MOVE THEM OUTSIDE THIS STUDIO INSTANCE NOW.")
print("Proceed only if you want a completely blank studio environment.")
print("\nType 'YES_DELETE_ALL' (case-sensitive) to confirm deletion and execute, then press Enter.")
print("Anything else will abort the operation.")

confirmation = input("Confirmation: ")

if confirmation.strip() == "YES_DELETE_ALL": # Case-sensitive comparison
    # Get the home/studio path
    # On Lightning AI, this is typically /teamspace/studios/this_studio
    HOME_PATH = Path.home()

    # Get the current notebook's filename
    # Use a more robust way to find the notebook file name, as __file__ is not always reliable in notebooks
    notebook_filename = "LightningAnxiety (1) (1) (2).ipynb" # Explicitly set your notebook filename here for reliability
    notebook_path = HOME_PATH / notebook_filename

    main_py_path = HOME_PATH / "main.py"

    # Define items to EXCLUDE from deletion
    EXCLUDE_LIST = [
        notebook_path,
        main_py_path
    ]

    print(f"\n--- Starting Comprehensive Deletion in {HOME_PATH} ---")
    deleted_count = 0
    skipped_count = 0

    for item in HOME_PATH.iterdir():
        # Convert item to absolute path for consistent comparison with EXCLUDE_LIST
        abs_item = item.resolve()
        if abs_item in EXCLUDE_LIST:
            print(f"ℹ️ Skipping protected item: {item.name}")
            skipped_count += 1
            continue

        print(f"🗑️ Attempting to delete: {item.name} ({item})...")
        try:
            if item.is_dir():
                shutil.rmtree(item)
            else:
                item.unlink() # Delete file
            print(f"✅ Successfully deleted: {item.name}")
            deleted_count += 1
        except Exception as e:
            print(f"❌ Error deleting {item.name} ({item}): {e}")

    print("\n--- Comprehensive Cleanup Process Complete ---")
    print(f"Summary: {deleted_count} items deleted, {skipped_count} items skipped (protected).")
    print("Please restart your runtime and run the notebook from the first cell for a fresh start.")
else:
    print("\nOperation aborted. No folders were deleted.\n")

# --- Ensure we are back in BASE_DIR at the end of the cell ---
if os.getcwd() != str(BASE_DIR):
    os.chdir(BASE_DIR)
# --- End of Directory Management ---
